In [ ]:
import pandas as pd
import numpy as np


In [ ]:

def eval(pred_path,true_path):
    '''
    Reads the Predicted Recommendations and Compares it with the Actual Purchases in the Testing Period
    Arguments : 
        pred_path - Path to the File containing the Generated Recommendations
        true_path - Path to the File containing the Actual Purchases during the testing week
    
    '''
  rec = pd.read_csv(pred_path)
  true = pd.read_csv(true_path)
  rec = rec[rec.customer_id.isin(true.customer_id)]
  df = pd.DataFrame(true.groupby("customer_id")['article_id'].apply(list))
  df['customer_id']  = df.index
  df = df.rename(columns = {'article_id':'relevant_items'})
  df = df.reset_index(drop=True)
  whole_data = rec.merge(df,on="customer_id")

  k = []

  for i in whole_data['article_id']:
    k.append(i.strip('][').split(','))
  
  l = []

  for i in whole_data['relevant_items']:
    #l.append(i)
    l.append(str(i).strip('][').split(','))
  print(np.mean([mapk(l,k,k=i) for i in range(1,13) ]))

In [ ]:
def apk(actual, predicted, k=12):
    # Computes Average Precision at K
    if len(predicted)>k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    #predicted = predicted.split()
    #actual = actual.split()
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    #Computes Mean Average Precision at K 
 
    return np.mean([apk(a,p,k) for a,p in zip(actual,predicted)])

In [9]:
print("MAP@12 Using Item Similarity Approach Implemented through turicreate",eval("turicreate_item_similarity_recommender.csv","transactions_test_subset.csv"))

MAP@12 Using Item Similarity Approach Implemented through turicreate 0.01871074801814001


In [8]:
print("MAP@12 Using Content Based  ",eval("content_based_recommender.csv","transactions_test_subset.csv"))

MAP@12 Using Content Based   0.021639737008585853


In [2]:
print("MAP@12 Using Ranking Factorization  ",eval("ranking_factorization_recommender.csv","transactions_test_subset.csv"))

MAP@12 Using Ranking Factorization   0.027764412212098896


In [1]:
print("MAP@12 Using ALS Collaborative",eval("test_predictions_ALS.csv","transactions_test_subset.csv"))

MAP@12 Using ALS Collaborative 0.025400162713870553


In [3]:
print("MAP@12 Using Item Item Collaborative",eval("test_predictions_collab.csv","transactions_test_subset.csv"))

MAP@12 Using Item Item Collaborative 0.02273702200656474
